# LLM Optimization Modelling Experiment

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [ ]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Generate the mathematical model

In [ ]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [ ]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Employee Management

**Parameters:**

* **S_target:** Target sales units (5500)
* **S_FTH:** Sales per hour for full-time employees (5 units/hour)
* **S_PTH:** Sales per hour for part-time employees (2 units/hour)
* **H_FT:** Target hours for full-time employees (800 hours)
* **H_PT:** Target hours for part-time employees (320 hours)
* **OT_FT_target:** Target overtime hours for full-time employees (100 hours)
* **P1:** Priority factor for sales deviation
* **P2:** Priority factor for full-time overtime deviation
* **P3:** Priority factor for full-time hours deviation (doubled)
* **P4:** Priority factor for overtime deviation (general)
* **C_OT_PT:** Relative cost of part-time overtime (3 times full-time overtime)

**Decision Variables:**

* **S_dev:** Deviation from the sales target (positive or negative)
* **H_FT_dev:** Deviation from full-time employee target hours (positive or negative)
* **H_PT_dev:** Deviation from part-time employee target hours (positive or negative)
* **OT_FT_dev:** Deviation from full-time overtime target hours (positive or negative)
* **OT_FT:** Total full-time overtime hours
* **OT_PT:** Total part-time overtime hours 

**Objective Function:**
Minimize the weighted sum of deviations:

```
Minimize: 
    P1 * |S_dev| + 
    P2 * |OT_FT_dev| + 
    2 * P3 * |H_FT_dev| + 
    P3 * |H_PT_dev| + 
    P4 * (OT_FT + C_OT_PT * OT_PT) 
```

**Constraints:**

* **Sales Constraint:**
    *  S_target + S_dev = S_FTH * (H_FT + H_FT_dev + OT_FT) + S_PTH * (H_PT + H_PT_dev + OT_PT)
* **Full-time Overtime Constraint:**
    * OT_FT = OT_FT_target + OT_FT_dev
* **Non-negativity Constraints:**
    * OT_FT >= 0
    * OT_PT >= 0

**Explanation:**

* The objective function minimizes the weighted impact of all deviations, considering their priorities and relative costs.
* The sales constraint ensures that the total sales (including deviations) are calculated based on employee hours (including deviations and overtime).
* The full-time overtime constraint connects the total full-time overtime hours with the target and deviation.
* The non-negativity constraints ensure that overtime hours cannot be negative.

This model allows you to find the optimal balance between your sales goals, employee hours, and overtime, considering your specific priorities and the relative cost of part-time overtime. By adjusting the priority factors and relative costs, you can customize the model to reflect the unique needs and preferences of your business. 


## 3. Generate the pyomo code

In [ ]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [ ]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
S_target = 5500  # Target sales units
S_FTH = 5  # Sales per hour for full-time employees
S_PTH = 2  # Sales per hour for part-time employees
H_FT = 800  # Target hours for full-time employees
H_PT = 320  # Target hours for part-time employees
OT_FT_target = 100  # Target overtime hours for full-time employees
P1 = 1  # Priority factor for sales deviation
P2 = 2  # Priority factor for full-time overtime deviation
P3 = 1  # Priority factor for full-time/part-time hours deviation
P4 = 0.5  # Priority factor for overtime deviation (general)
C_OT_PT = 3  # Relative cost of part-time overtime 

# Create a model
model = pyo.ConcreteModel()

# Decision Variables
model.S_dev = pyo.Var(within=pyo.Reals)  # Deviation from the sales target
model.H_FT_dev = pyo.Var(within=pyo.Reals)  # Deviation from full-time employee target hours
model.H_PT_dev = pyo.Var(within=pyo.Reals)  # Deviation from part-time employee target hours
model.OT_FT_dev = pyo.Var(within=pyo.Reals)  # Deviation from full-time overtime target hours
model.OT_FT = pyo.Var(within=pyo.NonNegativeReals)  # Total full-time overtime hours
model.OT_PT = pyo.Var(within=pyo.NonNegativeReals)  # Total part-time overtime hours

# Objective Function
model.obj = pyo.Objective(expr= P1 * abs(model.S_dev) +
                              P2 * abs(model.OT_FT_dev) +
                              2 * P3 * abs(model.H_FT_dev) +
                              P3 * abs(model.H_PT_dev) +
                              P4 * (model.OT_FT + C_OT_PT * model.OT_PT),
                          sense=pyo.minimize)

# Constraints
model.sales_constraint = pyo.Constraint(expr=S_target + model.S_dev == S_FTH * (H_FT + model.H_FT_dev + model.OT_FT) + 
                                              S_PTH * (H_PT + model.H_PT_dev + model.OT_PT))

model.overtime_constraint = pyo.Constraint(expr=model.OT_FT == OT_FT_target + model.OT_FT_dev)

# Solve the model
solver = pyo.SolverFactory('cbc') # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Sales Deviation (S_dev): ", model.S_dev.value)
print("Full-Time Hours Deviation (H_FT_dev): ", model.H_FT_dev.value)
print("Part-Time Hours Deviation (H_PT_dev): ", model.H_PT_dev.value)
print("Full-Time Overtime Deviation (OT_FT_dev): ", model.OT_FT_dev.value)
print("Total Full-Time Overtime (OT_FT): ", model.OT_FT.value)
print("Total Part-Time Overtime (OT_PT): ", model.OT_PT.value)
``` 

This code defines the model, variables, objective function, and constraints as described in your mathematical formulation. It uses sample data for the parameters, which you can replace with your actual data. Finally, it solves the model using the CBC solver (you can use other solvers) and prints the optimal values for the decision variables. 


## 4. Input problem data and try running the generated code

In [1]:
%%capture
import sys
import os
if 'google.colab' in sys.modules:
  !pip install idaes-pse --pre
  !idaes get-extensions --to ./bin
  os.environ['PATH'] += ':bin'

In [2]:
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
S_target = 5500  # Target sales units
S_FTH = 5  # Sales per hour for full-time employees
S_PTH = 2  # Sales per hour for part-time employees
H_FT = 800  # Target hours for full-time employees
H_PT = 320  # Target hours for part-time employees
OT_FT_target = 100  # Target overtime hours for full-time employees
P1 = 1  # Priority factor for sales deviation
P2 = 2  # Priority factor for full-time overtime deviation
P3 = 3  # Priority factor for full-time/part-time hours deviation
P4 = 4  # Priority factor for overtime deviation (general)
C_OT_PT = 3  # Relative cost of part-time overtime

# Create a model
model = pyo.ConcreteModel()

# Decision Variables
model.S_dev = pyo.Var(within=pyo.Reals)  # Deviation from the sales target
model.H_FT_dev = pyo.Var(within=pyo.Reals)  # Deviation from full-time employee target hours
model.H_PT_dev = pyo.Var(within=pyo.Reals)  # Deviation from part-time employee target hours
model.OT_FT_dev = pyo.Var(within=pyo.Reals)  # Deviation from full-time overtime target hours
model.OT_FT = pyo.Var(within=pyo.NonNegativeReals)  # Total full-time overtime hours
model.OT_PT = pyo.Var(within=pyo.NonNegativeReals)  # Total part-time overtime hours

# Objective Function
model.obj = pyo.Objective(expr= P1 * abs(model.S_dev) +
                              P2 * abs(model.OT_FT_dev) +
                              2 * P3 * abs(model.H_FT_dev) +
                              P3 * abs(model.H_PT_dev) +
                              P4 * (model.OT_FT + C_OT_PT * model.OT_PT),
                          sense=pyo.minimize)

# Constraints
model.sales_constraint = pyo.Constraint(expr=S_target + model.S_dev == S_FTH * (H_FT + model.H_FT_dev + model.OT_FT) +
                                              S_PTH * (H_PT + model.H_PT_dev + model.OT_PT))

model.overtime_constraint = pyo.Constraint(expr=model.OT_FT == OT_FT_target + model.OT_FT_dev)

# Solve the model
solver = pyo.SolverFactory('couenne') # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Sales Deviation (S_dev): ", model.S_dev.value)
print("Full-Time Hours Deviation (H_FT_dev): ", model.H_FT_dev.value)
print("Part-Time Hours Deviation (H_PT_dev): ", model.H_PT_dev.value)
print("Full-Time Overtime Deviation (OT_FT_dev): ", model.OT_FT_dev.value)
print("Total Full-Time Overtime (OT_FT): ", model.OT_FT.value)
print("Total Part-Time Overtime (OT_PT): ", model.OT_PT.value)

Optimal Solution:
Sales Deviation (S_dev):  -360.0
Full-Time Hours Deviation (H_FT_dev):  0.0
Part-Time Hours Deviation (H_PT_dev):  0.0
Full-Time Overtime Deviation (OT_FT_dev):  0.0
Total Full-Time Overtime (OT_FT):  100.0
Total Part-Time Overtime (OT_PT):  0.0


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [ ]:
print(response.text)

## Mathematical Optimization Model for Employee Management

**Parameters:**

* **S_target:** Target sales units (5500)
* **S_FTH:** Sales per hour for full-time employees (5 units/hour)
* **S_PTH:** Sales per hour for part-time employees (2 units/hour)
* **H_FT:** Target hours for full-time employees (800 hours)
* **H_PT:** Target hours for part-time employees (320 hours)
* **OT_FT_target:** Target overtime hours for full-time employees (100 hours)
* **P1:** Priority factor for sales deviation
* **P2:** Priority factor for full-time overtime deviation
* **P3:** Priority factor for full-time hours deviation (doubled)
* **P4:** Priority factor for overtime deviation (general)
* **C_OT_PT:** Relative cost of part-time overtime (3 times full-time overtime)

**Decision Variables:**

* **S_dev:** Deviation from the sales target (positive or negative)
* **H_FT_dev:** Deviation from full-time employee target hours (positive or negative)
* **H_PT_dev:** Deviation from part-time employee target 

In [ ]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
S_target = 5500  # Target sales units
S_FTH = 5  # Sales per hour for full-time employees
S_PTH = 2  # Sales per hour for part-time employees
H_FT = 800  # Target hours for full-time employees
H_PT = 320  # Target hours for part-time employees
OT_FT_target = 100  # Target overtime hours for full-time employees
P1 = 1  # Priority factor for sales deviation
P2 = 2  # Priority factor for full-time overtime deviation
P3 = 1  # Priority factor for full-time/part-time hours deviation
P4 = 0.5  # Priority factor for overtime deviation (general)
C_OT_PT = 3  # Relative cost of part-time overtime 

# Create a model
model = pyo.ConcreteModel()

# Decision Variables
model.S_dev = pyo.Var(within=pyo.Reals)  # Deviation from the sales target
model.H_FT_dev = pyo.Var(within=pyo.Reals)  # Deviation from full-time employee target hours
model.H_PT_dev = pyo.Var(within=pyo.Reals)  # Deviation from part-tim